<a href="https://colab.research.google.com/github/kyo-ke/m_learn/blob/master/col_gogh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install -U -q PyDrive

     |████████████████████████████████| 993kB 36.6MB/s 


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

def st_drive():
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    return drive


def file_list(drive,dir_id):
    file_list = drive.ListFile({'q': "'%s' in parents and trashed=false" % dir_id}).GetList()
    for f in file_list:
        print("name: " + f["title"] + ", id: " + f["id"])
    return


def use_file(drive,dir_id,file_name):
    file_list = drive.ListFile({'q': "'%s' in parents and trashed=false" % dir_id}).GetList()
    file_id = 'nothing'
    for f in file_list:
        if f['title'] == file_name:
            file_id = f['id']
    drive_file = drive.CreateFile({'id': file_id})
    drive_file.GetContentFile(file_name)
    return file_name


In [0]:
n_drive = st_drive()

In [0]:
dir_id = '12JW1M933ErGRinXgFbD0wjiTdxu0pzhv'
n_vgg = 'VGG_ILSVRC_16_layers.caffemodel'
n_o_img = 't_skate.png'
n_s_img = 'test_style.jpg'


In [6]:
use_file(n_drive,dir_id,n_vgg)
use_file(n_drive,dir_id,n_o_img)
use_file(n_drive,dir_id,n_s_img)


'test_style.jpg'

In [0]:


import chainer
from chainer import cuda
import chainer.functions as F
import chainer.links as L
from chainer.links import caffe
from chainer import Variable, optimizers


import os
import sys

import numpy as np
from PIL import Image


import pickle


In [0]:

class VGG:
    def __init__(self, fn="VGG_ILSVRC_16_layers.caffemodel", alpha=[0,0,1,1], beta=[1,1,1,1]):
        print ("load model... %s"%fn)
        self.model = caffe.CaffeFunction(fn)
        self.alpha = alpha
        self.beta = beta
    def forward(self, x):
        y1 = self.model['conv1_2'](F.relu(self.model['conv1_1'](x)))
        x1 = F.average_pooling_2d(F.relu(y1), 2, stride=2)
        y2 = self.model['conv2_2'](F.relu(self.model['conv2_1'](x1)))
        x2 = F.average_pooling_2d(F.relu(y2), 2, stride=2)
        y3 = self.model['conv3_3'](F.relu(self.model['conv3_2'](F.relu(self.model['conv3_1'](x2)))))
        x3 = F.average_pooling_2d(F.relu(y3), 2, stride=2)
        y4 = self.model['conv4_3'](F.relu(self.model['conv4_2'](F.relu(self.model['conv4_1'](x3)))))
    #    x4 = F.average_pooling_2d(F.relu(y4), 2, stride=2)
    #    y5 = model.conv5_3(F.relu(model.conv5_2(F.relu(model.conv5_1(x4)))))
        return [y1,y2,y3,y4]


def subtract_mean(x0):
    x = x0.copy()
    x[0,0,:,:] -= 120
    x[0,1,:,:] -= 120
    x[0,2,:,:] -= 120
    return x
def add_mean(x0):
    x = x0.copy()
    x[0,0,:,:] += 120
    x[0,1,:,:] += 120
    x[0,2,:,:] += 120
    return x



In [0]:
def image_resize(img_file, width):
    gogh = Image.open(img_file)
    orig_w, orig_h = gogh.size[0], gogh.size[1]
    if orig_w>orig_h:
        new_w = width
        new_h = width*orig_h//orig_w
        gogh = np.asarray(gogh.resize((new_w,new_h)))[:,:,:3].transpose(2, 0, 1)[::-1].astype(np.float32)
        gogh = gogh.reshape((1,3,new_h,new_w))
        print("image resized to: ", gogh.shape)
        hoge= np.zeros((1,3,width,width), dtype=np.float32)
        hoge[0,:,width-new_h:,:] = gogh[0,:,:,:]
        gogh = subtract_mean(hoge)
    else:
        new_w = width*orig_w//orig_h
        new_h = width
        gogh = np.asarray(gogh.resize((new_w,new_h)))[:,:,:3].transpose(2, 0, 1)[::-1].astype(np.float32)
        gogh = gogh.reshape((1,3,new_h,new_w))
        print("image resized to: ", gogh.shape)
        hoge= np.zeros((1,3,width,width), dtype=np.float32)
        hoge[0,:,:,width-new_w:] = gogh[0,:,:,:]
        gogh = subtract_mean(hoge)
    return xp.asarray(gogh), new_w, new_h


In [0]:
def save_image(img, width, new_w, new_h, it):
    def to_img(x):
        im = np.zeros((new_h,new_w,3))
        im[:,:,0] = x[2,:,:]
        im[:,:,1] = x[1,:,:]
        im[:,:,2] = x[0,:,:]
        def clip(a):
            return 0 if a<0 else (255 if a>255 else a)
        im = np.vectorize(clip)(im).astype(np.uint8)
        Image.fromarray(im).save(args.out_dir+"/im_%05d.png"%it)

    if args.gpu>=0:
        img_cpu = add_mean(img.get())
    else:
        img_cpu = add_mean(img)
    if width==new_w:
        to_img(img_cpu[0,:,width-new_h:,:])
    else:
        to_img(img_cpu[0,:,:,width-new_w:])


In [0]:
def get_matrix(y):
    ch = y.data.shape[1]
    wd = y.data.shape[2]
    gogh_y = F.reshape(y, (ch,wd**2))
    gogh_matrix = F.matmul(gogh_y, gogh_y, transb=True)/np.float32(ch*wd**2)
    return gogh_matrix


In [0]:
class Clip(chainer.Function):
    def forward(self, x):
        x = x[0]
        ret = cuda.elementwise('T x','T ret','ret = x<-120?-120:(x>136?136:x);','clip')(x)
        return ret


In [0]:
def generate_image(img_orig, img_style, width, nw, nh, max_iter, lr, img_gen=None):
    mid_orig = nn.forward(Variable(img_orig))
    style_mats = [get_matrix(y) for y in nn.forward(Variable(img_style))]

    if img_gen is None:
        if args.gpu >= 0:
            img_gen = xp.random.uniform(-20,20,(1,3,width,width),dtype=np.float32)
    img_gen = chainer.links.Parameter(img_gen)
    optimizer = optimizers.Adam(alpha=lr)
    optimizer.setup(img_gen)
    for i in range(max_iter):
        img_gen.zerograds()

        x = img_gen.W
        y = nn.forward(x)

        L = Variable(xp.zeros((), dtype=np.float32))
        for l in range(len(y)):
            ch = y[l].data.shape[1]
            wd = y[l].data.shape[2]
            gogh_y = F.reshape(y[l], (ch,wd**2))
            gogh_matrix = F.matmul(gogh_y, gogh_y, transb=True)/np.float32(ch*wd**2)

            L1 = np.float32(args.lam) * np.float32(nn.alpha[l])*F.mean_squared_error(y[l], Variable(mid_orig[l].data))
            L2 = np.float32(nn.beta[l])*F.mean_squared_error(gogh_matrix, Variable(style_mats[l].data))/np.float32(len(y))
            L += L1+L2

            if i%1000==0:
                print(i,l,L1.data,L2.data)

        L.backward()
        img_gen.W.grad = x.grad
        optimizer.update()

        tmp_shape = x.data.shape
        if args.gpu >= 0:
            img_gen.W.data += Clip().forward(img_gen.W.data).reshape(tmp_shape) - img_gen.W.data
        else:
            def clip(x):
                return -120 if x<-120 else (136 if x>136 else x)
            img_gen.W.data += np.vectorize(clip)(img_gen.W.data).reshape(tmp_shape) - img_gen.W.data

        if (i%1000==0 or i == (max_iter - 1)): 
            save_image(img_gen.W.data, W, nw, nh, i)


In [0]:

class args:
    def __init__(self):
        self.moedel = 'vgg'
        self.orig_img = n_o_img
        self.style_img = n_s_img
        self.out_dir = '/content/drive/My Drive/data'
        self.gpu = 0
        self.iter = 10000
        self.lr = 4.0
        self.lam = 0.09
        self.width = 435


In [0]:
args = args()

In [0]:

if args.gpu >= 0:
    cuda.check_cuda_available()
    chainer.Function.type_check_enable = False
    cuda.get_device(args.gpu).use()
    xp = cuda.cupy


In [17]:
nn = VGG()

load model... VGG_ILSVRC_16_layers.caffemodel


In [0]:
if args.gpu>=0:
    nn.model.to_gpu()


In [0]:
W = args.width

In [20]:
img_content,nw,nh = image_resize(args.orig_img, W)

image resized to:  (1, 3, 290, 435)


In [21]:
img_style,_,_ = image_resize(args.style_img, W)

image resized to:  (1, 3, 435, 326)


In [0]:
generate_image(img_content, img_style, W, nw, nh, img_gen=None, max_iter=args.iter, lr=args.lr)

0 0 0.0 180192.56
0 1 0.0 447550.4
0 2 26179.91 109104.84
0 3 1581.0887 428.37683
1000 0 0.0 152.51236
1000 1 0.0 470.78763
1000 2 10786.259 2221.6406
1000 3 385.16492 27.999


In [0]:
!ls

adc.json  sample_data  t_skate.png
drive	  style_2.png  VGG_ILSVRC_16_layers.caffemodel


In [0]:
!ls drive

'My Drive'


In [0]:
out_image_dir = '/content/drive/My Drive/out_images'
out_model_dir = '/content/drive/My Drive/out_models'
